[![Binder](https://mybinder.org/badge_logo.svg)](https://gesis.mybinder.org/v2/gh/homalg-project/CapAndHomalgNotebooks/master?urlpath=git-pull%3Frepo%3Dhttps%253A%252F%252Fgithub.com%252Fhomalg-project%252FLoopIntegrals%26urlpath%3Dtree%252FLoopIntegrals%252Fexamples%252F%252Fnotebooks%252F1LoopBox.ipynb%26branch%3Dmaster)

<img src="oneloopbox.png" alt="drawing" width="300" alt="centered image"/>

We consider the example of the massless one-loop box with a single loop momentum $\ell_1$ and four onshell external momenta $k_1,\ldots,k_4$ satsifying $k_i^2 = 0$ and $\sum_{i=1}^4 k_i = 0$ (the latter due to conservation of momentum):
$$
  I(a_1, \ldots, a_4) =
  \int
  \operatorname{d}^D \ell_1
  \frac{1}{\big[\underbrace{-\ell_1^2}_{P_1}\big]^{a_1} \big[\underbrace{-(\ell_1-k_1)^2}_{P_2}\big]^{a_2} \big[\underbrace{-(\ell_1-k_1-k_2)^2}_{P_3}\big]^{a_3} \big[\underbrace{-(\ell_1+k_4)^2}_{P_4}\big]^{a_4} }
$$

The integral depends on the space-time dimension $D$ and the kinematic invariants $s_{12} := 2 k_1 \cdot k_2$ and $s_{14} := 2 k_1 \cdot k_4$.

It is well-known that this integral family has a basis consisting of three master integerals which can be chosen as
$$
  \{ I(1,1,1,1), I(1,0,1,0), I(0,1,0,1) \} \mbox{.}
$$
This is also the set produced by Laporta's algorithm.
Our approach produces another basis
$$
  \{ I(1,1,1,1), I(1,1,0,1), I(1,1,1,0) \} \mbox{.}
$$

This notebook includes the computations of [1, Examples 5.2 and 6.5].

[1] Mohamed Barakat, Robin Brüser, Claus Fieker, Tobias Huber, Jan Piclum, Feynman integral reduction using Gröbner bases, [(arXiv:2210.05347)](https://arxiv.org/abs/2210.05347).

In [1]:
using CapAndHomalg

CapAndHomalg v1.4.7
Imported OSCAR's components GAP and Singular_jll
Type: ?CapAndHomalg for more information


The $\mathsf{GAP}$ package $\mathtt{LoopIntegrals}$ computes IBP relations among loop integrals. It relies on
* the computer algebra system <font style="font-variant: small-caps">Singular</font> for commutative Gröbner bases in polynomial rings;
* its subsystem <font style="font-variant: small-caps">Plural</font> for noncommutative Gröbner bases in the double shift algebra with polynomial coefficients;
* Chyzak's $\mathsf{Maple}$ package <TT>Ore_algebra</TT> for noncommutative Gröbner bases in the double shift algebra with __rational__ coefficients;
* the Julia package <font style="font-variant: small-caps">Hecke</font> for the linear algebra computations over the field of rational functions.

In [2]:
LoadPackage( "LoopIntegrals" )

We instantiate the loop diagram by specifying the loop momenta (here $\ell_1$) and the linear independent external momenta (here chosen to be $k_1, k_2, k_4$, recall $k_3 = -k_1-k_2-k_4$):

In [3]:
LD = LoopDiagram( "l1", "k1..2,k4" )

GAP: <A loop diagram with loop momenta [ l1 ] & external momenta [ k1, k2, k4 ]>

We encode the onshell condition for the independent external momenta $k_1, k_2, k_4$, namely $k_i^2 = 0$ for $i=1,2,4$:

In [4]:
rel1 = List( ExternalMomenta( LD ), k -> k^2 )

GAP: [ k1^2, k2^2, k4^2 ]

Then we encode onshell condition for the dependent external momentum: $k_3$, namely $k_3^2 = 0 = (k_1+k_2+k_4)^2$:

In [5]:
rel2 = GapObj([ (k1+k2+k4)^2 ]);

We now add both relations to our loop diagram data structure:

In [6]:
SetRelationsOfExternalMomenta( LD, Concatenation( rel1, rel2 ) )

The above relations among the external momenta imply the existence of two independent kinematic variables, the first of which is $s_{12} = 2 k_1 \cdot k_2$:

In [7]:
s12 = 2*k1*k2

GAP: 2*k1*k2

The following command instructs the system to replace any occurance of $2 k_1 \cdot k_2$ by the symbol $s_{12}$:

In [8]:
SetAbbreviation( s12, "s12" )

We repeat the above for the second kinematic variable $s_{14} = 2 k_1 \cdot k_4$:

In [9]:
s14 = 2*k1*k4

GAP: 2*k1*k4

In [10]:
SetAbbreviation( s14, "s14" )

With the above relations among the external momenta and the abbreviations of the independent kinematic variables the software is now able to express any polynomial in the scalar products of the external momenta in terms of the kinematic variables. This is the so-called subalgebra membership problem.

We declare the four propagators $P_1 = -\ell_1^2, P_2 = -(\ell_1-k_1)^2, P_3 = -(\ell_1-k_1-k_2)^2, P_4 = -(\ell_1+k_4)^2$:

In [11]:
SetPropagators( LD, -[ l1^2, (l1-k1)^2, (l1-k1-k2)^2, (l1+k4)^2 ] )

In this examples the loop integral has no numerators:

In [12]:
SetNumerators( LD, -[ ] )

It remains to declare the remaing independent Lorentz invariants:

In [13]:
SetIndependentLorentzInvariants( LD, [ l1^2, l1*k1, l1*k2, l1*k4, s12, s14 ] )

and indicate which of them are the kinematic ones:

In [14]:
SetExtraLorentzInvariants( LD, [ s12, s14 ] )

The polynomial ring $R$ associated to the loop diagram is a polynomial ring in the propagator variables $D_1, D_2, D_3, D_4$ over the polyomial ring in the dimension $D$ and the kinematic variables $s_{12}, s_{14}$:
$$
  \mathbb{Q}[D,s_{12},s_{14}][D_1,D_2,D_3,D_4] \mbox{.}
$$

In [15]:
R = RingOfLoopDiagram( LD )

GAP: Q[D,s12,s14][D1,D2,D3,D4]

The following computations will take place in the associated double-shift algebra
$$
  Y^\mathrm{pol} := \mathbb{Q}[D,s_{12},s_{14}][a_1,a_2,a_3,a_4]<D_1,D_1^-,D_2,D_2^-,D_3,D_3^-,D_4,D_4^-> / ( D_i D_i^- = 1 = D_i^- D_i \mid i = 1, \ldots, 4 ) \mbox{.}
$$

This a free associate algebra in the indeterminates $a_1, \ldots, a_4, D_1, \ldots, D_4, D_1^-, \ldots, D_4^-$ over the polynomial ring $\mathbb{Q}[D,s_{12},s_{14}]$ satisfying the relations:

* $D_i a_i = (a_i - 1) D_i$, $D_i^- a_i = (a_i + 1) D_i^-$, for all $i=1, \ldots, 4$;
* $a_i a_j = a_j a_i$, $D_i D_j = D_j D_i$, $D_i^- D_j^- = D_j^- D_i^-$ for all $i,j = 1, \ldots, 4$;
* $D_i a_j = a_j D_i$, $D_i^- a_j = a_j D_i^-$, for all $i \neq j = 1, \ldots, 4$;
* $D_i D_i^- = D_i^- D_i$ for all $i = 1, \ldots, 4$;
* $D_i D_i^- = 1$.


In [16]:
Ypol = DoubleShiftAlgebra( R )

GAP: Q[D,s12,s14][a1,a2,a3,a4]<D1,D1_,D2,D2_,D3,D3_,D4,D4_>/( D1*D1_-1, D2*D2_-1, D3*D3_-1, D4*D4_-1 )

In [17]:
R = RingOfLoopDiagram( LD )

GAP: Q[D,s12,s14][D1,D2,D3,D4]

In [18]:
Ypol = DoubleShiftAlgebra( R )

GAP: Q[D,s12,s14][a1,a2,a3,a4]<D1,D1_,D2,D2_,D3,D3_,D4,D4_>/( D1*D1_-1, D2*D2_-1, D3*D3_-1, D4*D4_-1 )

In [19]:
ibps = MatrixOfIBPRelations( LD )

GAP: <A 4 x 1 matrix over a residue class ring>

In [20]:
r1 = ibps[1,1]

GAP: |[ -a2*D1*D2_-s12*a3*D3_-a3*D1*D3_-a4*D1*D4_+D-2*a1-a2-a3-a4 ]|

In [21]:
ViewList( DecomposeInMonomials( r1 ) )

[ [ |[ -a2 ]|, |[ D1*D2_ ]| ],
  [ |[ -a3 ]|, |[ D1*D3_ ]| ],
  [ |[ -a4 ]|, |[ D1*D4_ ]| ],
  [ |[ -s12*a3 ]|, |[ D3_ ]| ],
  [ |[ D-2*a1-a2-a3-a4 ]|, |[ 1 ]| ] ]


In [22]:
Gpol = BasisOfRows( ibps )

GAP: <A non-zero 28 x 1 matrix over a residue class ring>

In [23]:
NormalForm( "a1*D1_" /  Ypol, Gpol )

GAP: |[ a1*D1_ ]|

In [24]:
gen = GeneratorsOfScalelessSectors( LD )

GAP: <A 1 x 4 matrix over an external ring>

In [25]:
Display( gen )

D3*D4,D1*D4,D2*D3,D1*D2


Now we look at the special IBP relations (cf. [1, Example 6.5]):

In [26]:
E12 = PairOfMatricesOfLoopDiagramInPropagators( LD )

GAP: [ <A 4 x 4 matrix over an external ring>, <A 4 x 4 matrix over an external ring> ]

In [27]:
Display( E12[1] )

2*D1,     D1-D2,     -s12+D2-D3,-D1+D4,    
D1+D2,    D1-D2,     D2-D3,     s14-D1+D4, 
s12+D1+D3,s12+D1-D2, D2-D3,     -s12-D1+D4,
D1+D4,    -s14+D1-D2,s14+D2-D3, -D1+D4     


In [28]:
S = SyzygiesOfColumns( E12 )

GAP: <A non-zero 4 x 12 matrix over an external ring>

In [29]:
Sred = ReducedBasisOfColumnModule( S )

GAP: <A non-zero 4 x 6 matrix over an external ring>

In [30]:
Display( Sred )

D2-D4,D1-D3,s12*D4+2*D3*D4-2*D4^2,     s14*D3-2*D3^2+2*D3*D4,-D3*D4^2,D3^2*D4, 
D4,   -D1,  -s12*D4+D2*D4-D3*D4+2*D4^2,-D1*D3-D3*D4,         D3*D4^2, 0,       
0,    -D1,  D1*D4+D2*D4,               -2*D1*D3,             0,       D1*D3*D4,
D2,   0,    2*D2*D4,                   -D1*D3-D2*D3,         D2*D3*D4,0        


In [31]:
Display( EntriesOfHomalgMatrixAsListList( CertainColumns( Sred, Array( 1:3 ) ) ) )

[ [ D2-D4, D1-D3, s12*D4+2*D3*D4-2*D4^2 ], [ D4, -D1, -s12*D4+D2*D4-D3*D4+2*D4^2 ], [ 0, -D1, D1*D4+D2*D4 ], [ D2, 0, 2*D2*D4 ] ]


In [32]:
Sibp1 = IBPRelation( CertainColumns( Sred, [ 1 ] ), LD )

GAP: |[ -s14*a2+s14*a4+D*D2-a1*D2-a2*D2-a3*D2-a4*D2-D*D4+a1*D4+a2*D4+a3*D4+a4*D4 ]|

In [33]:
ViewList( DecomposeInMonomials( Sibp1 ) )

[ [ |[ D-a1-a2-a3-a4 ]|, |[ D2 ]| ],
  [ |[ -D+a1+a2+a3+a4 ]|, |[ D4 ]| ],
  [ |[ -s14*a2+s14*a4 ]|, |[ 1 ]| ] ]


In [34]:
Sibp2 = IBPRelation( CertainColumns( Sred, [ 2 ] ), LD )

GAP: |[ -s12*a1+s12*a3+D*D1-a1*D1-a2*D1-a3*D1-a4*D1-D*D3+a1*D3+a2*D3+a3*D3+a4*D3 ]|

In [35]:
ViewList( DecomposeInMonomials( Sibp2 ) )

[ [ |[ D-a1-a2-a3-a4 ]|, |[ D1 ]| ],
  [ |[ -D+a1+a2+a3+a4 ]|, |[ D3 ]| ],
  [ |[ -s12*a1+s12*a3 ]|, |[ 1 ]| ] ]


In [36]:
Sibp3 = IBPRelation( CertainColumns( Sred, [ 3 ] ), LD )

GAP: |[ -s12*s14*a4-s14*a4*D1-s12*a4*D2-s14*a4*D3+D*s12*D4-2*s12*a2*D4-2*s14*a2*D4-2*s12*a3*D4-s12*a4*D4+2*s14*a4*D4+2*D*D3*D4-2*a1*D3*D4-2*a2*D3*D4-2*a3*D3*D4-2*a4*D3*D4-2*D*D4^2+2*a1*D4^2+2*a2*D4^2+2*a3*D4^2+2*a4*D4^2+s12*s14+s14*D1+s12*D2+s14*D3+s12*D4-2*s14*D4+2*D3*D4-2*D4^2 ]|

In [37]:
ViewList( DecomposeInMonomials( Sibp3 ) )

[ [ |[ 2*D-2*a1-2*a2-2*a3-2*a4+2 ]|, |[ D3*D4 ]| ],
  [ |[ -2*D+2*a1+2*a2+2*a3+2*a4-2 ]|, |[ D4^2 ]| ],
  [ |[ -s14*a4+s14 ]|, |[ D1 ]| ],
  [ |[ -s12*a4+s12 ]|, |[ D2 ]| ],
  [ |[ -s14*a4+s14 ]|, |[ D3 ]| ],
  [ |[ D*s12-2*s12*a2-2*s14*a2-2*s12*a3-s12*a4+2*s14*a4+s12-2*s14 ]|, 
  |[ D4 ]| ],
  [ |[ -s12*s14*a4+s12*s14 ]|, |[ 1 ]| ] ]


In [38]:
SGpol = BasisOfSpecialIBPRelations( LD )

GAP: <A non-zero 28 x 1 matrix over a residue class ring>

In [39]:
Gpol == SGpol

true